# Setting
Caution: the entire process of 1000x1000 analysis files will take approximately 4 hours or more.

If you want to spend less time, please use the 100x100 file.

In [ ]:
import pandas as pd
import matplotlib
import os
import sys
from scipy import stats
import pickle as pkl 
! pip install pycaret
! pip install numpy==1.20.0
import numpy as np
from pycaret.regression import *
import matplotlib.pyplot as plt
import math 
! pip install geopandas
import geopandas
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from sklearn.preprocessing import RobustScaler
! pip install osmnx
! pip install plot
! pip install matplotlib==3.1.3
import osmnx as ox
import networkx as nx
import plotly.graph_objects as go
from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.20.3-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.4 MB)
  Using cached matplotlib-3.6.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.4 MB)
     |████████████████████████████████| 282 kB 8.6 MB/s 
  Using cached yellowbrick-1.4-py3-none-any.whl (274 kB)
     |████████████████████████████████| 271 kB 53.4 MB/s 
     |████████████████████████████████| 269 kB 64.7 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
  Attempting uninstall: yellowbrick
    Found existing installation: yellowbrick 1.3.post1
    Uninstalling yellowbrick-1.3.post1:
      Successfully uninstall

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.20.0-cp38-cp38-manylinux2010_x86_64.whl (15.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas 1.5.2 requires numpy>=1.20.3; python_version < "3.10", but you have numpy 1.20.0 which is incompatible.
osmnx 1.2.2 requires numpy>=1.22, but you have numpy 1.20.0 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.8 which is incompatible.
cmdstanpy 1.0.8 requires numpy>=1.21, but you have numpy 1.20.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.23.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.0
    Uninstalling numpy-1.20.0:
      Successfully uninstalled numpy-1.20.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 1.23.5 which is incompatible.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 2.3.8 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached matplotlib-3.1.3-cp38-cp38-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.6.2
    Uninstalling matplotlib-3.6.2:
      Successfully uninstalled matplotlib-3.6.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.5.0 requires matplotlib<3.7,>=3.2, but you have matplotlib 3.1.3 which is incompatible.
osmnx 1.2.2 requires matplotlib>=3.5, but you have matplotlib 3.1.3 which is incompatible.


# Calculate and imput risk score to each node

In [ ]:
# give a weight to each data point according to crime's sentence, considering user's risk perception.
def give_weight(user_input, root_dir):
    with open(root_dir + "2018_ped_raw_data.pkl", "rb") as f:
        crime = pkl.load(f)
    with open(root_dir + "crime_weight_info.pkl", "rb") as f:
        crime_weight = pkl.load(f)
    
    input_list = [] # If user_input variable is empty, it will be calculated as default weight which is based on original crime sentence, without user's risk perception.
    
    if 0 in user_input:
        input_list.extend([3,4,5])
    if 1 in user_input:
        input_list.extend([0, 8])
    if 2 in user_input:
        input_list.extend([1])
    if 3 in user_input:
        input_list.extend([13, 20])
    if 4 in user_input:
        input_list.extend([12])
    if 5 in user_input:
        input_list.extend([6, 9, 16, 7])
    if 6 in user_input:
        input_list.extend([15])
    if 7 in user_input:
        input_list.extend([18])
    if 8 in user_input:
        input_list.extend([2, 10])
    if 9 in user_input:
        input_list.extend([14])
    if 10 in user_input:
        input_list.extend([17, 19])
        
    crime_weight.loc[input_list, "weight"] = 60
    crime_table = crime.merge(crime_weight, how='inner', on='type')
    longitude = []
    latitude = []

    for i in range(len(crime_table)):
        for item in range(crime_table['weight'][i]):
            longitude.append(crime_table['longitude'][i])
            latitude.append(crime_table['latitude'][i])

    crime_df = pd.DataFrame({'longitude':longitude, 'latitude':latitude})
    return crime_df

In [ ]:
# estimate the density using Gausian Kernel Density Estimation.
def get_f(location):
    x = location["longitude"]
    y = location["latitude"]

    xmin = -87.9
    xmax = -87.5
    ymin = 41.6
    ymax = 42.05

    print(xmin, xmax, ymin, ymax)
    xx, yy = np.mgrid[xmin:xmax:1000j, ymin:ymax:1000j]

    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = stats.gaussian_kde(values)

    f = np.reshape(kernel(positions).T, xx.shape)
    
    return f

In [ ]:
# normalize population effect
def population_normalization(crime_ex_df, root_dir):
    xmin = -87.9
    xmax = -87.5
    ymin = 41.6
    ymax = 42.05
    xx, yy = np.mgrid[xmin:xmax:1000j, ymin:ymax:1000j]

    with open(root_dir + "chicago_polygon_ratio.pkl", "rb") as f:
        chicago_polygon = pkl.load(f)
        
    chicago_location = chicago_polygon[['the_geom','zipcode','SHAPE_AREA']]
    vectors_list = []
    count = 0
    for vector_set in chicago_polygon['the_geom']:
        tmp = []
        vector_set = vector_set.replace("(","")
        vector_set = vector_set.replace(")","")
        for index, vector in enumerate(vector_set[13:].split(",")):
            if index == 0:
                longitude = vector.split(" ")[0]
                latitude = vector.split(" ")[1]
                tmp.append([float(longitude), float(latitude)])
            else:
                longitude = vector.split(" ")[1]
                latitude = vector.split(" ")[2]
                tmp.append([float(longitude), float(latitude)])
            
        count += 1
        vectors_list.append(tmp)

    for i in range(60):
        xs, ys = zip(*vectors_list[i])
     
    data_list = []
    data_list.append(np.squeeze((np.reshape(xx,(1000000,1)))).T)
    data_list.append(np.squeeze((np.reshape(yy,(1000000,1)))).T)
    data_list.append(np.squeeze((np.reshape(crime_ex_df,(1000000,1)))).T)
    crime_density_vertex = pd.DataFrame(data_list).T

    crime_density_vertex.columns = ['longitude','latitude','crime_density']
    print("# 3-1 session complete.")

    vertex_zipcode_list=  []
    zipcode_area_list = []
    ratio_list = []

    for vertex in crime_density_vertex.itertuples():
        x = vertex.longitude 
        y = vertex.latitude
        z = vertex.crime_density

        isbreak = False
        for polygon_vertex_list, zipcode, area, ratio in zip(vectors_list,chicago_polygon['zipcode'], chicago_polygon['SHAPE_AREA'],chicago_polygon['ratio_2018'] ):
            if Polygon(polygon_vertex_list).contains(Point(x,y)):
                vertex_zipcode_list.append(zipcode)
                zipcode_area_list.append(area)
                ratio_list.append(ratio)
                isbreak = True
                break

        if isbreak is False:
            vertex_zipcode_list.append(0)
            zipcode_area_list.append(0)
            ratio_list.append(0)

    crime_density_vertex['zipcode'] = vertex_zipcode_list
    crime_density_vertex['area'] = zipcode_area_list
    crime_density_vertex['ratio'] = ratio_list

    crime_density_vertex['population'] = 0
    crime_density_vertex

    crime_density_vertex['adjust_crime_density'] = 0

    for data_tuple in crime_density_vertex.itertuples():
        if data_tuple.ratio == 0:
            pass
        else:
            if data_tuple.crime_density / data_tuple.ratio != 0:
                crime_density_vertex.loc[data_tuple.Index, 'adjust_crime_density'] = data_tuple.crime_density / (data_tuple.ratio)
                crime_density_vertex.loc[data_tuple.Index, 'crime_density'] = data_tuple.crime_density
    final_chicago_crime = crime_density_vertex[crime_density_vertex['zipcode']!=0]

    return final_chicago_crime

In [ ]:
# searching nearst node of graph and cells to allocate the risk score.
def minimum_search (longitude, latitude, cell_longitude, cell_latitude):
    result = np.sqrt((cell_longitude - longitude)** 2 + (cell_latitude - latitude) ** 2)
    maxind = np.argmin(result)

    return maxind

In [ ]:
# main function, whole process. each step of process is described using print function

if __name__ == "__main__":  
    #argument = sys.argv
    #del argument[0]
    print("\n", "#0. Process Starts.")
    
    drive.mount('/content/drive') # need to be modified
    root_dir = '/content/drive/Shareddrives/RABBIT/Final_code/PyFile/' # need to be modified 

    with open(root_dir + "urban_5_dataset_1000by1000.pkl", "rb") as f:
        urban_dataset = pkl.load(f)
    with open(root_dir + "graph_nodes.pkl", "rb") as f:
        node = pkl.load(f)

    model = load_model(root_dir+"r_c_f_ET_0.99996")
    
    user_input = [1, 4] # please insert your risk perception
    
    print("\n", "#1. Give a weight...")
    
    crime_df = give_weight(user_input, root_dir)
    print(crime_df)
    
    print("\n", "#2. Get F value...")
    
    crime_ex_df = get_f(crime_df)
    print(crime_ex_df)
    
    print("\n", "#3. Population normalization...")
    
    chicago_crime = population_normalization(crime_ex_df, root_dir)
    print(chicago_crime) 
    
    print("\n", "#4. Data scailing...")
    
    urban_dataset["crime_2018"] = chicago_crime["adjust_crime_density"]
    features = ["shotspotter", "abd_building", "lightout", "support", "garbage_data", "crime_2018"]
    x = urban_dataset.loc[:, features].values
    x = RobustScaler().fit_transform(x)
    scailed_data = pd.DataFrame(x, columns=features)
    
    print(scailed_data)
    print("\n", "#5. Combine urban factor and crime to risk score (model predict)...")
    
    dataset = predict_model(model, scailed_data)
    
    dataset = dataset.reset_index(drop=True)
    dataset["longitude"] = chicago_crime["longitude"].reset_index(drop=True)
    dataset["latitude"] = chicago_crime["latitude"].reset_index(drop=True)
    print(dataset)
        
    print("\n", "#6. Assign node...")
        
    nearst_cell_index_list = []
    crime_score_list = []
    for pos_longitude, pos_latitude in zip(node['x'],node['y']):
        index = minimum_search(pos_longitude, pos_latitude, dataset['longitude'], dataset['latitude'])
        nearst_cell_index_list.append(dataset.iloc[index].name)
        crime_score_list.append(dataset.iloc[index]["Label"])
        
    node["assigned_cell"] = nearst_cell_index_list
    node["crime_score"] = crime_score_list
    print(node) # Weighted graph is generated.
    print("\n", "#0. Finish. We got a new node.")


 #0. Process Starts.
Mounted at /content/drive


INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/drive/Shareddrives/RABBIT/Final_code/PyFile/r_c_f_ET_0.99996, platform=None, authentication=None, verbose=True)


Transformation Pipeline and Model Successfully Loaded

 #1. Give a weight...


# Calculate and imput risk score to each edge
The risk score of the edge is the mean value of the risk scores in starting point and end point.



In [ ]:
node.rename(columns={"crime_score": "risk score"}, inplace=True)
nodes = node

In [ ]:
with open(root_dir + "graph_edges.pkl", "rb") as f:
    edges = pkl.load(f)

# reset the index of edges to 'u'
edges_u = edges.reset_index(level=1).reset_index(level=1)

# merge edges and nodes
edges_u = edges_u.merge(nodes['risk score'], how='inner', left_index=True, right_index=True)

In [ ]:
# add the 'v' to the index
edges_u.index.rename('u', inplace=True)
edges_u.set_index([edges_u.index,'v'], inplace=True)
edges_u.rename(columns={"risk score": "risk score_u"}, inplace=True)

In [ ]:
# reset the index of edges to 'v'
edges_v = edges_u.reset_index(level=0)

# merge edges and nodes
edges_v = edges_v.merge(nodes['risk score'], how='inner', left_index=True, right_index=True)

In [ ]:
# add the 'v' to the index
edges_v.rename(columns={"u": "v", "risk score" : "risk score_v"}, inplace=True)
edges_v.index.rename('u', inplace=True)
edges_v.set_index([edges_v.index,'v'], inplace=True)
edges_v.set_index([edges_v.index,'key'], inplace=True)

In [ ]:
# get the mean of the start and end nodes's riskiness
edges_v['risk score'] = (edges_v['risk score_u'] + edges_v['risk score_v'])/2

# Create new features in edge considering the length and riskiness to 3:7, 7:3
To return various routes for user, create the 2 more features that considers the length and riskiness in ratio of 3:7 and 7:3.

In [ ]:
# use Robust Scaler
transformer_length = RobustScaler()
transformer_length.fit(np.reshape(list(edges_v['length']),(-1,1)))

transformer_risk = RobustScaler()
transformer_risk.fit(np.reshape(list(edges_v['risk score']),(-1,1)))

In [ ]:
scaled_length = transformer_length.transform(np.reshape(list(edges_v['length']),(-1,1)))
scaled_risk = transformer_risk.transform(np.reshape(list(edges_v['risk score']),(-1,1)))

In [ ]:
edges_v['l_r_3_7'] = (scaled_length * 0.3 + scaled_risk * 0.7) - min((scaled_length * 0.3 + scaled_risk * 0.7))
edges_v['l_r_5_5'] = (scaled_length * 0.5 + scaled_risk * 0.5) - min((scaled_length * 0.5 + scaled_risk * 0.5))
edges_v['l_r_7_3'] = (scaled_length * 0.7 + scaled_risk * 0.3) - min((scaled_length * 0.7 + scaled_risk * 0.3))

edges = edges_v

# Re-create the graph based on the nodes and edges considering the riskiness

In [ ]:
# create graph with nodes and edges including risk score
G_risk = ox.graph_from_gdfs(nodes, edges)

# Find the routes using the shortest path algorithm

In [ ]:
# User input
orig = (41.87961687579612, -87.62370895911769) # User Input: Art Institute of Chicago
dest = (41.878747088475116, -87.64029358250872) # User Input: Union Station

# find the nearest node from the orig and dest
orig_node = ox.distance.nearest_nodes(G_risk, orig[1], orig[0])
dest_node = ox.distance.nearest_nodes(G_risk, dest[1], dest[0])

# apply the shortest path algorithm
# length:risk = 0:10
route0 = ox.shortest_path(G_risk, orig_node, dest_node, weight="risk score") #safest
# length:risk = 3:7
route1 = ox.shortest_path(G_risk, orig_node, dest_node, weight="l_r_3_7") 
# length:risk = 7:3
route2 = ox.shortest_path(G_risk, orig_node, dest_node, weight="l_r_7_3")
# length:risk = 10:0 
route3 = ox.shortest_path(G_risk, orig_node, dest_node, weight="length") #shortest

In [ ]:
# compare the four routes
route0_length = int(sum(ox.utils_graph.get_route_edge_attributes(G_risk, route0, "length")))
route1_length = int(sum(ox.utils_graph.get_route_edge_attributes(G_risk, route1, "length")))
route2_length = int(sum(ox.utils_graph.get_route_edge_attributes(G_risk, route2, "length")))
route3_length = int(sum(ox.utils_graph.get_route_edge_attributes(G_risk, route3, "length")))

route0_risk = int(sum(ox.utils_graph.get_route_edge_attributes(G_risk, route0, "risk score")))
route1_risk = int(sum(ox.utils_graph.get_route_edge_attributes(G_risk, route1, "risk score")))
route2_risk = int(sum(ox.utils_graph.get_route_edge_attributes(G_risk, route2, "risk score")))
route3_risk = int(sum(ox.utils_graph.get_route_edge_attributes(G_risk, route3, "risk score")))

## Results of 4 routes

In [ ]:
print("Route 0 is", route0_length, "meters and has", route0_risk, "risk score.")
print("Route 1 is", route1_length, "meters and has", route1_risk, "risk score.")
print("Route 2 is", route2_length, "meters and has", route2_risk, "risk score.")
print("Route 3 is", route3_length, "meters and has", route3_risk, "risk score.")

In [ ]:
# get the information about the nodes in route
route0_nodes = []
for node in route0:
  route0_nodes.append(list(nodes.loc[[node]].values[0]))

route1_nodes = []
for node in route1:
  route1_nodes.append(list(nodes.loc[[node]].values[0]))

route2_nodes = []
for node in route2:
  route2_nodes.append(list(nodes.loc[[node]].values[0]))

route3_nodes = []
for node in route3:
  route3_nodes.append(list(nodes.loc[[node]].values[0]))

In [ ]:
route0_nodes_df = pd.DataFrame(route0_nodes)
route1_nodes_df = pd.DataFrame(route1_nodes)
route2_nodes_df = pd.DataFrame(route2_nodes)
route3_nodes_df = pd.DataFrame(route3_nodes)

route0_nodes_df.columns = ['y', 'x', 'street_count', 'ref', 'highway', 'geometry', 'assigned_cell', 'risk score']
route1_nodes_df.columns = ['y', 'x', 'street_count', 'ref', 'highway', 'geometry', 'assigned_cell', 'risk score']
route2_nodes_df.columns = ['y', 'x', 'street_count', 'ref', 'highway', 'geometry', 'assigned_cell', 'risk score']
route3_nodes_df.columns = ['y', 'x', 'street_count', 'ref', 'highway', 'geometry', 'assigned_cell', 'risk score']

In [ ]:
# get the information about the edges in route
route0_edges = ox.utils_graph.get_route_edge_attributes(G_risk, route0)
route1_edges = ox.utils_graph.get_route_edge_attributes(G_risk, route1)
route2_edges = ox.utils_graph.get_route_edge_attributes(G_risk, route2)
route3_edges = ox.utils_graph.get_route_edge_attributes(G_risk, route3)

## Plot the results of 4 routes

### Plot on the folium map

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
routes = [route0, route1, route2, route3]
rc = ['green', 'blue', 'brown', 'red']
route_plot = ox.plot_graph_routes(G_risk, routes, route_colors=rc, route_linewidth=6, node_size=10, orig_dest_size=300, ax=ax)

In [ ]:
route_map = ox.plot_route_folium(G_risk, route0, popup_attribute="risk score", color='green', opacity=0.5)
route_map = ox.plot_route_folium(G_risk, route1, popup_attribute="risk score", color='blue', route_map=route_map, opacity=0.5)
route_map = ox.plot_route_folium(G_risk, route2, popup_attribute="risk score", color='brown', route_map=route_map, opacity=0.5)
route_map = ox.plot_route_folium(G_risk, route3, popup_attribute="risk score", color='red', route_map=route_map, opacity=0.5)

In [ ]:
route_map

### Plot on the colored map

In [ ]:
mean_y = (orig[0] + dest[0]) /2
mean_x = (orig[1] + dest[1]) /2
mean_point = mean_y + mean_x
bbox = ox.utils_geo.bbox_from_point((mean_y, mean_x), dist=4000)

In [ ]:
ec = ox.plot.get_edge_colors_by_attr(G_risk, "risk score", cmap="plasma", num_bins=10, equal_size=True)

In [ ]:
ox.plot_graph_routes(G_risk, routes, route_colors=rc, bbox=bbox, edge_color=ec, edge_linewidth=0.5, node_size=0, bgcolor="#e2e2e2")